# STEP17 メモリ管理と循環参照

- STEP1:これまでDeZeroの変数と関数を作った
- STEP2:関数としてSquareを作った
- STEP3:別の新しい関数を実装し複数の関数を組み合わせて計算を行う
- STEP4:数値微分でいったん微分を計算してみる
- STEP5:バックプロパゲーションの仕組み
- STEP6:VariableとFunctionクラスを拡張して、バックプロパゲーションを用いて微分を求められるように実装
- STEP7:順伝搬がどのような計算であっても自動的に逆伝搬を計算できるようにする, 具体的にはVariableクラスを拡張し使用した関数情報を保持できるようにする
- STEP8:処理効率の改善するために、backwardメソッドをwhileループに置き換える。Variable関数のみの書き換えでOK
- STEP9:pythonの関数として使えるようにする, y.grad=np.array(1.0)を省略する, ndarrayだけ扱う
- STEP10:DeepLearningのフレームワークのテスト方法について説明
- STEP11:関数に対して可変長入出力に対応する
- STEP12:11の拡張
- STEP13:逆伝搬に関しても関数に対して可変長入出力に対応する
- STEP14:同じ変数を繰り返し使うと発生する問題に対応する y = add(x, x)
- STEP15:さまざまなトポロジーの計算グラフに対応すること
- STEP16:さまざまなトポロジーの計算グラフに対応すること
- STEP17:パフォーマンス改善テクニック: pythonのメモリ管理について学ぶ, weakrefをいれ循環参照を防ぐ => メモリ改善

In [25]:
#DIR = "deep-learning-from-scratch-3/steps/"
#! diff $DIR/step16.py $DIR/step17.py -y

## 事前準備: 使用メモリ測定ライブラリー

- https://pyteyon.hatenablog.com/entry/2020/04/29/150000

In [ ]:
#pip install memory-profiler

## STEP17適用前のメモリ

### テストプログラムの作成 (改善前)

In [14]:
%%writefile test_step17_before.py
import numpy as np
from memory_profiler import profile

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output.grad for output in f.outputs]
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x

#@profile
class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = outputs
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


class Add(Function):
    def forward(self, x0, x1):
        y = x0 + x1
        return y

    def backward(self, gy):
        return gy, gy


def add(x0, x1):
    return Add()(x0, x1)


for i in range(1000):
    x = Variable(np.random.randn(100000))  # big data
    y = square(square(square(x)))

Overwriting test_step17_before.py


### テスト (改善前)

In [15]:
! ls -l test_step17_before.py
! mprof run test_step17_before.py
! mprof peak

-rw-r--r--  1 daisuke  staff  2586  7 31 13:19 test_step17_before.py
mprof: Sampling memory every 0.1s
running new process
running as a Python program...
Using last profile data.
mprofile_20220731131931.dat	539.438 MiB


### テストプログラムの作成 (改善後)

In [16]:
%%writefile test_step17_after.py
import weakref
import numpy as np

class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None
        self.generation = 0

    def set_creator(self, func):
        self.creator = func
        self.generation = func.generation + 1

    def cleargrad(self):
        self.grad = None

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = []
        seen_set = set()

        def add_func(f):
            if f not in seen_set:
                funcs.append(f)
                seen_set.add(f)
                funcs.sort(key=lambda x: x.generation)

        add_func(self.creator)

        while funcs:
            f = funcs.pop()
            gys = [output().grad for output in f.outputs]  # output is weakref
            gxs = f.backward(*gys)
            if not isinstance(gxs, tuple):
                gxs = (gxs,)

            for x, gx in zip(f.inputs, gxs):
                if x.grad is None:
                    x.grad = gx
                else:
                    x.grad = x.grad + gx

                if x.creator is not None:
                    add_func(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, *inputs):
        xs = [x.data for x in inputs]
        ys = self.forward(*xs)
        if not isinstance(ys, tuple):
            ys = (ys,)
        outputs = [Variable(as_array(y)) for y in ys]

        self.generation = max([x.generation for x in inputs])
        for output in outputs:
            output.set_creator(self)
        self.inputs = inputs
        self.outputs = [weakref.ref(output) for output in outputs]
        return outputs if len(outputs) > 1 else outputs[0]

    def forward(self, xs):
        raise NotImplementedError()

    def backward(self, gys):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.inputs[0].data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


for i in range(1000):
    x = Variable(np.random.randn(100000))  # big data
    y = square(square(square(x)))

Overwriting test_step17_after.py


### テスト (改善後)

In [17]:
! ls -l test_step17_after.py
! mprof run test_step17_after.py
! mprof peak

-rw-r--r--  1 daisuke  staff  2429  7 31 13:19 test_step17_after.py
mprof: Sampling memory every 0.1s
running new process
running as a Python program...
Using last profile data.
mprofile_20220731131942.dat	75.484 MiB


# 補足

## 参照カウント

https://www.sejuku.net/blog/90518

プログラム中でオブジェクトへの参照がない場合に、そのオブジェクトの割当を解除（メモリを開放）します。
ここでいう参照カウントとは「そのオブジェクトが参照されている数」を記録した数字です。これはオブジェクトごとに用意されています。
参照カウントが増えるのは以下の処理を行ったときです。

- 代入演算子を使ったとき: b = a (bはaを参照している)
- 引数渡しをしたとき: f(a)
- オブジェクトをコンテナ型オブジェクトに追加したとき: test_list.append(a)

参照カウントが1以上のとき、メモリは確保されたままになります。逆に0になったとき、GCはそれを不要だと判断します。
参照カウントについては`sys.getrefcount()`で実際に確認することができます。これはobject の参照数を返します。 object は (一時的に) getrefcount() からも参照されるため、参照数は予想される数よりも 1 多くなります。


```python
sys.getrefcount(object)¶
```


### 参照カウントの基本的な挙動

STEP3-1が4になる理由は以下の通り

1. 変数a自身の参照
2. bがaを参照
3. aがf関数の引数
4. Pythonの関数スタックが参照

In [19]:
import sys
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)


a = obj();        print("STEP1:",   sys.getrefcount(a) - 1)
b = a;            print("STEP2:",   sys.getrefcount(a) - 1)
f(a);             print("STEP3-2:", sys.getrefcount(a) - 1)
test_list = [a];  print("STEP4:",   sys.getrefcount(a) - 1)

# Noneにしても、0にならないことに注意
a = None;         print("STEP5:",   sys.getrefcount(a) - 1)

STEP1: 1
STEP2: 2
STEP3-1: 4
STEP3-2: 2
STEP4: 3
STEP5: 33706


### 参照カウント方式のメモリ管理 (getrefcountを利用する場合、教科書どおりの挙動を観測できない)

In [21]:
import sys
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

a = obj()
b = obj()
c = obj()

print(f"STEP0 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

a.b = b;          print(f"STEP1 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")
b.c = c;          print(f"STEP2 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")
a = b = c = None; print(f"STEP3 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

print(sys.getrefcount(None))

STEP0 a: 1, b: 1, c: 1
STEP1 a: 1, b: 2, c: 1
STEP2 a: 1, b: 2, c: 2
STEP3 a: 33727, b: 33727, c: 33727
33730


### そこで aのみ `a = None` とし、部分的に挙動を確認する

`a = None` とすると bの参照カウントが1となる

![](https://docs.google.com/drawings/d/e/2PACX-1vShs9FAFOfb9Fswa6MWy2Qj5pR6Qv3MajQuj7Z31SHGvQMcmz39J9mCEYvFK3PvKCM5sGaQx7LNYj3a/pub?w=162&h=149)

In [1]:
import sys
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

a = obj()
b = obj()
c = obj()


a.b = b
b.c = c
a = None
print(f"a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

print(sys.getrefcount(None))

a: 33715, b: 1, c: 2
33695


## 循環参照

### 教科書どおり動かしても観測できない

In [22]:
import sys
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

a = obj()
b = obj()
c = obj()

print(f"STEP0 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

a.b = b;          print(f"STEP1 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")
b.c = c;          print(f"STEP2 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")
c.a = a;          print(f"STEP3 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")
a = b = c = None; print(f"STEP4 a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

print(sys.getrefcount(None))

STEP0 a: 1, b: 1, c: 1
STEP1 a: 1, b: 2, c: 1
STEP2 a: 1, b: 2, c: 2
STEP3 a: 2, b: 2, c: 2
STEP4 a: 33744, b: 33744, c: 33744
33747


### aのみを `a=None` とすることで 循環参照を観測してみる

bの参照カウントが2となり、aの参照が無くなっていないことが分かる

In [23]:
import sys
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

a = obj()
b = obj()
c = obj()


a.b = b
b.c = c
c.a = a

print(f"a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

a = None

print(f"a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

print(sys.getrefcount(None))

a: 2, b: 2, c: 2
a: 33710, b: 2, c: 2
33713


## weakref実験

### weakrefなし

In [27]:
import sys
import weakref
import numpy as np
a = np.array([1, 2, 3])
b = a
print("b:", b)
#print("b():", b())
a = None
print("b(after a = none):", b)

b: [1 2 3]
b(after a = none): [1 2 3]


### weakrefあり

In [28]:
import sys
import weakref
import numpy as np
a = np.array([1, 2, 3])
b = weakref.ref(a)
print("b:", b)
print("b():", b())
a = None
print("b(after a = none):", b)
print("b()(after a = none):", b())

b: <weakref at 0x104c812c0; to 'numpy.ndarray' at 0x103ef6b10>
b(): [1 2 3]
b(after a = none): <weakref at 0x104c812c0; dead>
b()(after a = none): None


### 循環参照があるときのweakrefの挙動

`b`のカウントが`2`ではなくて`1`になっていることを確認できる

In [36]:
import sys
import weakref
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

a = obj()
b = obj()
c = obj()

#a.b = b
a.b = weakref.ref(b)
b.c = c
c.a = a

print(f"a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

a = None

print(f"a: {sys.getrefcount(a) - 1}, b: {sys.getrefcount(b) - 1}, c: {sys.getrefcount(c) - 1}")

print(sys.getrefcount(None))

a: 2, b: 1, c: 2
a: 36918, b: 1, c: 2
36921


In [75]:
import sys
import weakref
class obj:
    pass

def f(x):
    print("STEP3-1:", sys.getrefcount(x) - 1)

f = obj()
v = obj()

f.v = weakref.ref(v)
#f.v = v
v.f = f

print(f"f: {sys.getrefcount(f) - 1}, v: {sys.getrefcount(v) - 1}")

v = None

print(f"f: {sys.getrefcount(f) - 1}, v: {sys.getrefcount(v) - 1}")

print(sys.getrefcount(None))

f: 2, v: 1
f: 1, v: 37436
37439
